In [3]:
#!pip install -U neo4j 
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.20.0-py3-none-any.whl size=280771 sha256=5070ef2892aacc700d3b7bc21acef0e59008ddb2dcd44a8f22b2fadc638a65d0
  Stored in directory: /Users/mariamcnally/Library/Caches/pip/wheels/d6/1d/48/3e3738e5da9e07926dbcb7f6488ff7cb463156ade0c9f60cc5
Successfully built neo4j


In [5]:
!pip install "/Users/mariamcnally/Downloads/neo4j-5.20.0.tar"

Processing /Users/mariamcnally/Downloads/neo4j-5.20.0.tar
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.20.0-py3-none-any.whl size=280771 sha256=d893c2ffda411002f6b7c3f9d44b4e958a645599901f4548cb3a3e8119744619
  Stored in directory: /Users/mariamcnally/Library/Caches/pip/wheels/ed/58/7f/0e263b8192fb836a8b290398eab0e9561a9f1bd70c8c719063
Successfully built neo4j
  Attempting uninstall: neo4j
    Found existing installation: neo4j 5.20.0
    Uninstalling neo4j-5.20.0:
      Successfully uninstalled neo4j-5.20.0


In [8]:
from neo4j import GraphDatabase, RoutingControl, basic_auth

In [2]:
import pandas as pd
import numpy as np

## Data Clean

In [3]:
df1 = pd.read_csv('Materials/2016_-_Cities_Emissions_Reduction_Targets_20240207.csv')
df2 = pd.read_csv('Materials/2016_-_Citywide_GHG_Emissions_20240207.csv')
df3 = pd.read_csv('Materials/2017_-_Cities_Community_Wide_Emissions.csv')
df4 = pd.read_csv('Materials/2017_-_Cities_Emissions_Reduction_Targets_20240207.csv')
df5 = pd.read_csv('Materials/2023_Cities_Climate_Risk_and_Vulnerability_Assessments_20240207.csv')

dataframes = [df1, df2, df3, df4, df5]

In [4]:
ac = 'Organisation Number'
ci = 'City'
org = 'Organisation'
co = 'Country'
c40 = 'C40 City'
em = 'Total emissions (metric tonnes CO2e)'
pop = 'City Population'
year = 'Reporting Year'

for df in dataframes:
    df.rename(columns={'Account No': ac, 'Organization Number': ac, 'Account number': ac,
                       'Organization Name': org, 'Organization': org,
                       'Country/Area': co,
                       'City Short Name': ci,
                       'Baseline emissions (metric tonnes CO2e)': em, 'Total City-wide Emissions (metric tonnes CO2e)': em,
                       'Population': pop, 'Current Population': pop,
                       'Reporting year': year,
                       'C40': c40}, inplace=True)

In [5]:
c40_to_bool = lambda x: False if pd.isnull(x) else (True if x == 'C40' else x)
for df in dataframes:
    df['C40 City'] = df['C40 City'].apply(c40_to_bool)

## Setup connection

# DB URI and authentication
URI = "bolt://localhost:7687"
AUTH = ("neo4j","12345678")